In [1]:
import pandas as pd

customers_df = pd.read_csv('../data/hm/customers.csv')
articles_df = pd.read_csv('../data/hm/articles.csv', dtype={'article_id': str})

hi
hi!
2136454
hi!!


In [ ]:
article_df = articles_df[['article_id', 'prod_name', 'product_type_name']]
customers_df = customers_df[['customer_id', 'age']]
transactions_df = pd.read_csv('../data/hm/transactions_train.csv', dtype={'article_id': str})

In [71]:
df = pd.merge(transactions_df, article_df, how='inner', on='article_id')

customer_count = df.customer_id.value_counts()
valid_customers = customer_count[5000:15000]
valid_customer_ids = set(valid_customers.index)

valid_customer_mask = df.customer_id.isin(valid_customer_ids)

a = df.article_id.value_counts()
too_popular_articles = set(a[:50].index)
too_unpopular_articles = set(a[a < 20].index)

exclude_article_mask = df.article_id.isin(too_popular_articles | too_unpopular_articles)

mask = (valid_customer_mask & (~exclude_article_mask))

filtered_df = df[mask]

In [72]:
from tqdm import tqdm
tqdm.pandas()

raw_sequences = []
seq_labels = None

for customer_id, personal_df in tqdm(filtered_df.groupby('customer_id')):
    raw_sequences.append(list(personal_df.article_id.values))

100%|██████████| 10000/10000 [00:00<00:00, 16956.45it/s]


In [74]:
filtered_articles = filtered_df.article_id.unique()

In [75]:
item_list = []
filtered_articles_df = articles_df[article_df.article_id.isin(set(filtered_articles))]

for index, row in filtered_articles_df.iterrows():
    item_name = row.prod_name + ' - ' + row.product_type_name
    item_list.append(item_name)

In [76]:
raw_sequences = [' '.join(sequence) for sequence in raw_sequences]

In [77]:
sequence_df = pd.DataFrame(raw_sequences, columns=['sequence'])
sequence_df.to_csv('../data/hm/filtered_purchase_history.csv')

In [78]:
item_df = pd.DataFrame(item_list, columns=['name'])
item_df['article_id'] = filtered_articles
item_df.to_csv('../data/hm/items.csv')